# C2. Barrier Option in Libor Market Model 
# Barrier cap/floor

## Barrier Caplet Pricing Algorithms
Compare:
1. Analytic closed‑form (\~V)  
2. Algorithm 2.1 (weak 1, kick‑back)  
3. Algorithm 2.2 (weak ½, no VR)  
=> Sensitivity analysis
4. Algorithm 2.2 with variance reduction

$$
V_{\mathrm{caplet}}(t)
= \delta \, P\bigl(t,\,T_{i+1}\bigr)
\;\mathbb{E}^{\mathbb{Q}^{T_{i+1}}}
\Bigl[\bigl(L^i(T_i)-K\bigr)_+\;\chi\{\theta>T_i\}\,\bigm|\;\mathcal{F}_t\Bigr]
$$

#### Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd
import plotly.graph_objects as go
from tqdm import tqdm
from scipy.stats import norm
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.colors as mcolors
import copy

In [3]:
L0, H, K = 0.13, 0.28, 0.01
sigma, T = 0.25, 9.0
N_paths = 300000
h_list  = [0.25, 0.125, 0.0625, 0.03125, 0.015625] #[0.25, 0.125, 0.0625]

## Analytic closed‑form 

In [4]:
def analytic_vtilde_caplet(L, H, K, sigma, tau):
    v = sigma * np.sqrt(tau)
    def d_plus(x):  return (np.log(x) + 0.5*v**2)/v
    def d_minus(x): return (np.log(x) - 0.5*v**2)/v

    term1 = L*(norm.cdf(d_plus(L/K)) - norm.cdf(d_plus(L/H)))
    term2 = -K*(norm.cdf(d_minus(L/K)) - norm.cdf(d_minus(L/H)))
    term3 = -H*(norm.cdf(d_plus(H*H/(K*L))) - norm.cdf(d_plus(H/L)))
    term4 = K*L/H*(norm.cdf(d_minus(H*H/(K*L))) - norm.cdf(d_minus(H/L)))
    return term1 + term2 + term3 + term4

In [5]:
exact = analytic_vtilde_caplet(L0, H, K, sigma, T)
exact

0.06571345192162645

# Algorithm 2.1 – weak order 1 with kick‑back reflection

In [56]:
def simulate_algo1(L0, H, K, sigma, T, h, N_paths, seed=42):
    """
    Algorithm 2.1 (weak 1, kick‐back reflection).
    Returns (price, stderr, mean_exit_time).
    """
    rng = np.random.default_rng(seed)
    M = int(np.ceil(T / h))
    lnH = np.log(H)
    sqrt_h = np.sqrt(h)
    half_sigma2_h = 0.5 * sigma**2 * h
    # λ√h from (4.12)
    lambda_sqrt_h = -half_sigma2_h + sigma*sqrt_h

    payoffs = np.zeros(N_paths)
    exit_steps = np.zeros(N_paths, dtype=int)

    for i in tqdm(range(N_paths), desc="MC paths"):
        lnL = np.log(L0)
        for k in range(M):
            if lnL >= lnH + half_sigma2_h - sigma*sqrt_h:
                # in boundary zone
                delta = lnH - lnL
                p = lambda_sqrt_h / (delta + lambda_sqrt_h)
                if rng.random() < p:
                    # knock‐out at barrier
                    exit_steps[i] = k
                    lnL = lnH
                    break
                else:
                    # kick back in
                    lnL -= lambda_sqrt_h
                    xi = 1 if rng.random() < 0.5 else -1
                    lnL += -half_sigma2_h + sigma*sqrt_h*xi
            else:
                # usual Euler step
                xi = 1 if rng.random() < 0.5 else -1
                lnL += -half_sigma2_h + sigma*sqrt_h*xi
        else:
            # survived to maturity
            exit_steps[i] = M
            payoffs[i] = max(np.exp(lnL) - K, 0.0)

    price = payoffs.mean()
    stderr = payoffs.std(ddof=1) / np.sqrt(N_paths)
    mean_exit_time = exit_steps.mean() * h
    return price, stderr, mean_exit_time

In [57]:
results_algo1 = []

for h in h_list:
    p, se, _   = simulate_algo1(L0, H, K, sigma, T, h, N_paths)
    results_algo1.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

results_algo1

MC paths: 100%|██████████| 300000/300000 [01:07<00:00, 4471.69it/s]


[{'h': 0.25, 'Price': 0.06642984801006661, 'CI': 0.00020717463849089288},
 {'h': 0.125, 'Price': 0.06511021749444856, 'CI': 0.00020139090362803727},
 {'h': 0.0625, 'Price': 0.06566506450742415, 'CI': 0.00020297308288983186},
 {'h': 0.03125, 'Price': 0.06560974722134243, 'CI': 0.00020281099380507322},
 {'h': 0.015625, 'Price': 0.06571280756669262, 'CI': 0.00020316197326154662}]

## Algorithm 2.2 – weak order 1/2, absorbing barrier (no VR)

In [59]:
def simulate_algo2(L0, H, K, sigma, T, h, N_paths, seed=42):
    rng = np.random.default_rng(seed)
    M = int(np.ceil(T/h))
    lnH, sqrt_h = np.log(H), np.sqrt(h)
    half_sig2h = 0.5 * sigma**2 * h
    payoffs = np.zeros(N_paths)
    for i in tqdm(range(N_paths), desc="MC paths"):
        lnL = np.log(L0)
        for k in range(M):
            if lnL >= lnH + half_sig2h - sigma*sqrt_h:
                break
            xi = 1 if rng.random() < 0.5 else -1
            lnL += -half_sig2h + sigma*sqrt_h*xi
        payoffs[i] = max(np.exp(lnL)-K, 0.0) if k == M-1 else 0.0
    price, stderr = payoffs.mean(), payoffs.std(ddof=1)/np.sqrt(N_paths)
    return price, stderr

In [60]:
results_algo2 = []

for h in h_list:
    p, se = simulate_algo2(L0, H, K, sigma, T, h, N_paths)
    results_algo2.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

results_algo2

MC paths: 100%|██████████| 300000/300000 [01:06<00:00, 4542.36it/s]


[{'h': 0.25, 'Price': 0.06257531070865202, 'CI': 0.00020400285018166406},
 {'h': 0.125, 'Price': 0.06299697305907437, 'CI': 0.00020131107854003962},
 {'h': 0.0625, 'Price': 0.06321458719352833, 'CI': 0.00019953209213402877},
 {'h': 0.03125, 'Price': 0.06410563900060298, 'CI': 0.00020093403136147898},
 {'h': 0.015625, 'Price': 0.06450418164996352, 'CI': 0.0002018366595446813}]

In [85]:
# Prepare figure
fig = go.Figure()

# Add trace for Algorithm 1
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Price'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    name='Algorithm 1 - weak order 1 <br> with kick-back reflection'
))

# Add trace for Algorithm 2
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Price'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    name='Algorithm 2 - weak order 1/2 <br> with absorbing barrier'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo2),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact'
))

fig.update_layout(
    template="simple_white",
    title={
        "text": (
            # main title in default size:
            "Caplet price from Algorithm 1 vs Algorithm 2 with varying step size<br>"
            # parameters in a smaller span (e.g. 12px):
            f"<span style='font-size:12px;'>"
            f"L0={L0}, H={H}, K={K}, σ={sigma}, T={T}, "
            f"N_paths={N_paths}"
            "</span>"
        ),
        "x": 0.5,
        "xanchor": "center"
    }
)

fig.show()

From the plot above, it’s evident that Algorithm 1 (kick-back reflection) vastly outperforms Algorithm 2 (absorbing barrier). Even at the coarsest time step (h=0.25), Algorithm 1’s estimates lie almost exactly on the true caplet price (dotted line) and its confidence intervals remain uniformly small, reflecting its higher weak order and stability. By contrast, Algorithm 2 is heavily biased downward—starting near 0.0645—and only inches toward the exact value as h decreases. 

The familiar oscillatory pattern of the binary-tree scheme is visible in Algorithm 1’s curve but fades as the mesh is refined, whereas Algorithm 2’s convergence is monotonic yet slow. 

**In short, Algorithm 1 delivers superior accuracy and greater efficiency; Algorithm 2 requires prohibitively small steps to even approach the correct price.**

## Sensitivity analysis
Impact of changing L0, H, K, σ, T - one parameter at a time

In [199]:
## Normal color
# get the default Seaborn palette (first color is blue)
blue_rgb = sns.color_palette()[0]

# convert (r,g,b) floats → hex string
blue_hex = mcolors.to_hex(blue_rgb)

orange_rgb = sns.color_palette()[1]
orange_hex = mcolors.to_hex(orange_rgb)

def lighten_color(hex_color, amount=0.5):
    """
    Lighten a given color by mixing it with white.
    amount=0   → original color
    amount=1   → white
    """
    # convert hex to (r, g, b)
    r, g, b = mcolors.to_rgb(hex_color)
    # blend each channel toward 1 (white)
    r_l = r + (1 - r) * amount
    g_l = g + (1 - g) * amount
    b_l = b + (1 - b) * amount
    return mcolors.to_hex((r_l, g_l, b_l))

## Lighten
blue_pastel_hex = lighten_color(blue_hex, amount=0.4)
orange_pastel_hex = lighten_color(orange_hex, amount=0.4)


In [217]:
N_paths = 30_000
h_list = [0.25, 0.0625, 0.015625]

# perturbed values (one at a time)
L0, H, K = 0.13, 0.28, 0.01
sigma, T = 0.25, 9.0
L0_, H_, K_ = 0.08, 0.20, 0.005
sigma_, T_ = 0.30, 7.0

In [198]:
results_algo2 = []
results_algo2_ = []
results_algo1 = []
results_algo1_ = []

for h in h_list:
    print(h, "\n")
    p, se = simulate_algo2(L0, H, K, sigma, T, h, N_paths)
    results_algo2.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se = simulate_algo2(L0_, H, K, sigma, T, h, N_paths)
    results_algo2_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma, T, h, N_paths)
    results_algo1.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0_, H, K, sigma, T, h, N_paths)
    results_algo1_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

exact = analytic_vtilde_caplet(L0, H, K, sigma, T)
exact_ = analytic_vtilde_caplet(L0_, H, K, sigma, T)

0.25 



MC paths: 100%|██████████| 30000/30000 [00:00<00:00, 51163.36it/s]


0.0625 



MC paths: 100%|██████████| 30000/30000 [00:01<00:00, 15483.52it/s]


0.015625 



MC paths: 100%|██████████| 30000/30000 [00:07<00:00, 3825.11it/s]


In [200]:
# Prepare figure
fig = go.Figure()

###### Baselines
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo1),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Price'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    line=dict(color=blue_hex),
    marker=dict(color=blue_hex),
    name='Algorithm 1 - baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Price'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    line=dict(color=orange_hex),
    marker=dict(color=orange_hex),
    name='Algorithm 2 - baseline'
))

###### Perturbed

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact_]*len(results_algo1_),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],
    y=[d['Price'] for d in results_algo1_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1_], visible=True),
    line=dict(color=blue_pastel_hex),
    marker=dict(color=blue_pastel_hex),
    name='Algorithm 1 - perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2_],
    y=[d['Price'] for d in results_algo2_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2_], visible=True),
    line=dict(color=orange_pastel_hex),
    marker=dict(color=orange_pastel_hex),
    name='Algorithm 2 - perturbed'
))


###### General layout

fig.update_layout(
    template="simple_white",
    title={
        "text": (
            # main title in default size:
            "Caplet price sensitivity to L0<br>"
            # parameters in a smaller span (e.g. 12px):
            f"<span style='font-size:12px;'>"
            f"L0={L0} vs L0_={L0_} <br> (N_paths = {N_paths})"
            "</span>"
        ),
        "x": 0.5,
        "xanchor": "center"
    }
)

fig.show()

In [211]:
results_algo2 = []
results_algo2_ = []
results_algo1 = []
results_algo1_ = []

for h in h_list:
    print(h, "\n")
    p, se = simulate_algo2(L0, H, K, sigma, T, h, N_paths)
    results_algo2.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se = simulate_algo2(L0, H_, K, sigma, T, h, N_paths)
    results_algo2_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma, T, h, N_paths)
    results_algo1.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H_, K, sigma, T, h, N_paths)
    results_algo1_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

exact = analytic_vtilde_caplet(L0, H, K, sigma, T)
exact_ = analytic_vtilde_caplet(L0, H_, K, sigma, T)

# Prepare figure
fig = go.Figure()

###### Baselines
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo1),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Price'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    line=dict(color=blue_hex),
    marker=dict(color=blue_hex),
    name='Algorithm 1 - baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Price'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    line=dict(color=orange_hex),
    marker=dict(color=orange_hex),
    name='Algorithm 2 - baseline'
))

###### Perturbed

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact_]*len(results_algo1_),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],
    y=[d['Price'] for d in results_algo1_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1_], visible=True),
    line=dict(color=blue_pastel_hex),
    marker=dict(color=blue_pastel_hex),
    name='Algorithm 1 - perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2_],
    y=[d['Price'] for d in results_algo2_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2_], visible=True),
    line=dict(color=orange_pastel_hex),
    marker=dict(color=orange_pastel_hex),
    name='Algorithm 2 - perturbed'
))


###### General layout

fig.update_layout(
    template="simple_white",
    title={
        "text": (
            # main title in default size:
            "Caplet price sensitivity to H<br>"
            # parameters in a smaller span (e.g. 12px):
            f"<span style='font-size:12px;'>"
            f"H={H} vs H_={H_} <br> (N_paths = {N_paths})"
            "</span>"
        ),
        "x": 0.5,
        "xanchor": "center"
    }
)

fig.show()

0.25 



MC paths: 100%|██████████| 30000/30000 [00:00<00:00, 80034.35it/s]


0.0625 



MC paths: 100%|██████████| 30000/30000 [00:01<00:00, 21500.07it/s]


0.015625 



MC paths: 100%|██████████| 30000/30000 [00:05<00:00, 5494.15it/s]


In [212]:
results_algo2 = []
results_algo2_ = []
results_algo1 = []
results_algo1_ = []

for h in h_list:
    print(h, "\n")
    p, se = simulate_algo2(L0, H, K, sigma, T, h, N_paths)
    results_algo2.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se = simulate_algo2(L0, H, K_, sigma, T, h, N_paths)
    results_algo2_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma, T, h, N_paths)
    results_algo1.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K_, sigma, T, h, N_paths)
    results_algo1_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

exact = analytic_vtilde_caplet(L0, H, K, sigma, T)
exact_ = analytic_vtilde_caplet(L0, H, K_, sigma, T)

# Prepare figure
fig = go.Figure()

###### Baselines
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo1),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Price'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    line=dict(color=blue_hex),
    marker=dict(color=blue_hex),
    name='Algorithm 1 - baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Price'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    line=dict(color=orange_hex),
    marker=dict(color=orange_hex),
    name='Algorithm 2 - baseline'
))

###### Perturbed

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact_]*len(results_algo1_),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],
    y=[d['Price'] for d in results_algo1_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1_], visible=True),
    line=dict(color=blue_pastel_hex),
    marker=dict(color=blue_pastel_hex),
    name='Algorithm 1 - perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2_],
    y=[d['Price'] for d in results_algo2_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2_], visible=True),
    line=dict(color=orange_pastel_hex),
    marker=dict(color=orange_pastel_hex),
    name='Algorithm 2 - perturbed'
))


###### General layout

fig.update_layout(
    template="simple_white",
    title={
        "text": (
            # main title in default size:
            "Caplet price sensitivity to K<br>"
            # parameters in a smaller span (e.g. 12px):
            f"<span style='font-size:12px;'>"
            f"K={K} vs K_={K_} <br> (N_paths = {N_paths})"
            "</span>"
        ),
        "x": 0.5,
        "xanchor": "center"
    }
)

fig.show()

0.25 



MC paths: 100%|██████████| 30000/30000 [00:00<00:00, 64424.97it/s]


0.0625 



MC paths: 100%|██████████| 30000/30000 [00:01<00:00, 17130.10it/s]


0.015625 



MC paths: 100%|██████████| 30000/30000 [00:07<00:00, 4106.21it/s]


In [216]:
results_algo2 = []
results_algo2_ = []
results_algo1 = []
results_algo1_ = []

for h in h_list:
    print(h, "\n")
    p, se = simulate_algo2(L0, H, K, sigma, T, h, N_paths)
    results_algo2.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se = simulate_algo2(L0, H, K, sigma_, T, h, N_paths)
    results_algo2_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma, T, h, N_paths)
    results_algo1.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma_, T, h, N_paths)
    results_algo1_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

exact = analytic_vtilde_caplet(L0, H, K, sigma, T)
exact_ = analytic_vtilde_caplet(L0, H, K, sigma_, T)

# Prepare figure
fig = go.Figure()

###### Baselines
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo1),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Price'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    line=dict(color=blue_hex),
    marker=dict(color=blue_hex),
    name='Algorithm 1 - baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Price'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    line=dict(color=orange_hex),
    marker=dict(color=orange_hex),
    name='Algorithm 2 - baseline'
))

###### Perturbed

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact_]*len(results_algo1_),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],
    y=[d['Price'] for d in results_algo1_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1_], visible=True),
    line=dict(color=blue_pastel_hex),
    marker=dict(color=blue_pastel_hex),
    name='Algorithm 1 - perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2_],
    y=[d['Price'] for d in results_algo2_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2_], visible=True),
    line=dict(color=orange_pastel_hex),
    marker=dict(color=orange_pastel_hex),
    name='Algorithm 2 - perturbed'
))


###### General layout

fig.update_layout(
    template="simple_white",
    title={
        "text": (
            # main title in default size:
            "Caplet price sensitivity to sigma<br>"
            # parameters in a smaller span (e.g. 12px):
            f"<span style='font-size:12px;'>"
            f"sigma={sigma} vs sigma_={sigma_} <br> (N_paths = {N_paths})"
            "</span>"
        ),
        "x": 0.5,
        "xanchor": "center"
    }
)

fig.show()

0.25 



MC paths: 100%|██████████| 30000/30000 [00:00<00:00, 67339.79it/s]


0.0625 



MC paths: 100%|██████████| 30000/30000 [00:01<00:00, 18310.56it/s]


0.015625 



MC paths: 100%|██████████| 30000/30000 [00:06<00:00, 4370.72it/s]


In [218]:
results_algo2 = []
results_algo2_ = []
results_algo1 = []
results_algo1_ = []

for h in h_list:
    print(h, "\n")
    p, se = simulate_algo2(L0, H, K, sigma, T, h, N_paths)
    results_algo2.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se = simulate_algo2(L0, H, K, sigma, T_, h, N_paths)
    results_algo2_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma, T, h, N_paths)
    results_algo1.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma, T_, h, N_paths)
    results_algo1_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

exact = analytic_vtilde_caplet(L0, H, K, sigma, T)
exact_ = analytic_vtilde_caplet(L0, H, K, sigma, T_)

# Prepare figure
fig = go.Figure()

###### Baselines
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo1),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Price'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    line=dict(color=blue_hex),
    marker=dict(color=blue_hex),
    name='Algorithm 1 - baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Price'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    line=dict(color=orange_hex),
    marker=dict(color=orange_hex),
    name='Algorithm 2 - baseline'
))

###### Perturbed

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact_]*len(results_algo1_),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],
    y=[d['Price'] for d in results_algo1_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1_], visible=True),
    line=dict(color=blue_pastel_hex),
    marker=dict(color=blue_pastel_hex),
    name='Algorithm 1 - perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2_],
    y=[d['Price'] for d in results_algo2_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2_], visible=True),
    line=dict(color=orange_pastel_hex),
    marker=dict(color=orange_pastel_hex),
    name='Algorithm 2 - perturbed'
))


###### General layout

fig.update_layout(
    template="simple_white",
    title={
        "text": (
            # main title in default size:
            "Caplet price sensitivity to T<br>"
            # parameters in a smaller span (e.g. 12px):
            f"<span style='font-size:12px;'>"
            f"T={T} vs T_={T_} <br> (N_paths = {N_paths})"
            "</span>"
        ),
        "x": 0.5,
        "xanchor": "center"
    }
)

fig.show()

0.25 



MC paths: 100%|██████████| 30000/30000 [00:00<00:00, 75849.89it/s]


0.0625 



MC paths: 100%|██████████| 30000/30000 [00:01<00:00, 20212.13it/s]


0.015625 



MC paths: 100%|██████████| 30000/30000 [00:06<00:00, 4847.48it/s]


## Algorithm 2.2 with variance reduction (add Z from (4.6))

### Variance‑reduction term F(s,L) ≈ -σ ∂V/∂L from (4.7)

In [6]:
def F_vr_fd(L, H, K, sigma, tau, eps):
    """Finite-difference control-variate integrand."""
    Vp = analytic_vtilde_caplet(L + eps, H, K, sigma, tau)
    Vm = analytic_vtilde_caplet(L - eps, H, K, sigma, tau)
    dVdL = (Vp - Vm) / (2 * eps)
    return -sigma * dVdL

In [7]:
def F_vr_exact(L, H, K, sigma, tau, eps=None):
    """Exact closed-form ∂_L Ṽ from the barrier-caplet formula."""
    v = sigma * np.sqrt(tau)
    inv_v = 1.0 / v
    lnL = np.log(L)

    # deltas
    a1 = (lnL - np.log(K) + 0.5*v*v) * inv_v
    a2 = (lnL - np.log(H) + 0.5*v*v) * inv_v
    b1 = (lnL - np.log(K) - 0.5*v*v) * inv_v
    b2 = (lnL - np.log(H) - 0.5*v*v) * inv_v
    c1 = (np.log(H*H/(K*L)) + 0.5*v*v) * inv_v
    c2 = (np.log(H/L)    + 0.5*v*v) * inv_v
    d1 = (np.log(H*H/(K*L)) - 0.5*v*v) * inv_v
    d2 = (np.log(H/L)    - 0.5*v*v) * inv_v

    phi = norm.pdf
    Phi = norm.cdf

    T1 = (Phi(a1) - Phi(a2)) + inv_v*(phi(a1) - phi(a2))
    T2 = -(K/(L*v))*(phi(b1) - phi(b2))
    T3 =  (H/(L*v))*(phi(c1) - phi(c2))
    T4 = (K/H)*(Phi(d1) - Phi(d2)) - (K/(H*v))*(phi(d1) - phi(d2))

    dVdL = T1 + T2 + T3 + T4
    return -sigma * dVdL

In [8]:
def simulate_algo2(
    L0, H, K, sigma, T, h, N_paths,
    vr_method: str = "none",   # "none", "fd" or "exact"
    seed: int = 42,
    eps: float = 1e-4
):
    """
    Monte-Carlo Algo2 with choice of variance-reduction:

      vr_method="none"  → no control-variate
      vr_method="fd"    → finite-difference CV (needs eps)
      vr_method="exact" → exact CV (closed-form)
    """
    rng = np.random.default_rng(seed)
    M = int(np.ceil(T / h))
    lnH, sqrt_h = np.log(H), np.sqrt(h)
    half_sig2h = 0.5 * sigma**2 * h

    payoffs = np.zeros(N_paths)
    if vr_method != "none":
        Z_vals = np.zeros(N_paths)

    for i in tqdm(range(N_paths), desc="MC paths"):
        lnL = np.log(L0)
        Z = 0.0

        for k in range(M):
            tau = T - k*h
            # boundary check
            if lnL >= lnH + half_sig2h - sigma*sqrt_h:
                break

            xi = 1 if rng.random() < 0.5 else -1
            lnL += -half_sig2h + sigma*sqrt_h*xi

            if vr_method == "fd":
                Fvr = F_vr_fd(np.exp(lnL), H, K, sigma, tau, eps)
                Z += Fvr * sqrt_h * xi
            elif vr_method == "exact":
                Fvr = F_vr_exact(np.exp(lnL), H, K, sigma, tau)
                Z += Fvr * sqrt_h * xi

        payoff = max(np.exp(lnL) - K, 0.0) if k == M-1 else 0.0
        payoffs[i] = payoff
        if vr_method != "none":
            Z_vals[i] = Z

    # No control-variate: plain MC
    if vr_method == "none":
        price  = payoffs.mean()
        stderr = payoffs.std(ddof=1) / np.sqrt(N_paths)
        return price, stderr

    # Control-variate adjustment
    cov = np.cov(payoffs, Z_vals, ddof=1)[0,1]
    varZ = Z_vals.var(ddof=1)
    g    = cov / varZ
    Z_bar = Z_vals.mean()

    adj = payoffs - g*(Z_vals - Z_bar)
    price  = adj.mean()
    stderr = adj.std(ddof=1) / np.sqrt(N_paths)
    return price, stderr, g

In [10]:
h = 0.25

In [11]:
# 1) Plain MC
p0, e0 = simulate_algo2(L0, H, K, sigma, T, h, 3000, vr_method="none")

MC paths: 100%|██████████| 3000/3000 [00:00<00:00, 58346.33it/s]


In [224]:
# 2) FD-based CV
p_fd, e_fd, g_fd = simulate_algo2(
    L0, H, K, sigma, T, h, 3000,
    vr_method="fd", eps=1e-4
)

MC paths: 100%|██████████| 3000/3000 [12:36<00:00,  3.97it/s]


In [225]:
# 3) Exact-derivative CV
p_ex, e_ex, g_ex = simulate_algo2(
    L0, H, K, sigma, T, h, 3000,
    vr_method="exact"
)

MC paths: 100%|██████████| 3000/3000 [10:17<00:00,  4.86it/s]


In [226]:
print(f"plain: {p0:.6f} ± {e0:.6f}")
print(f"FD-CV:  {p_fd:.6f} ± {e_fd:.6f} (g={g_fd:.4f})")
print(f"Exact:  {p_ex:.6f} ± {e_ex:.6f} (g={g_ex:.4f})")

plain: 0.063772 ± 0.001016
FD-CV:  0.063772 ± 0.000932 (g=-0.0536)
Exact:  0.063772 ± 0.000932 (g=-0.0536)
